### Compare to the results from Royden_Husson_06_gji

In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.ThDSubduction0.RoydenHusson06 as RH06

#### Model setup and Variables

Vt - upper plate velocity 

Vm - lower mantle velocity

Vr - trench velocity

L - length of the slab 

lbd - thickness of the upper mantle

mu - viscosity

n - number of points along half the slab

![](./RH06_fig02.png)

In [ ]:
yr = 365.0 * 24.0 * 3600.0
cm_per_yr = 1e-2 / yr

# define variables
Vt = 0.0 * cm_per_yr
Vm = 0.0 * cm_per_yr
Vr = 1.0 * cm_per_yr
L = 1000e3
a = L / 2.0
lbd = 700e3
mu = 1e20
n_hs = 100


#### A solution for the Hele-Shaw flow field

At the wavelengths of interest for toroidal flowaround the subduction system as awhole, horizontal velocities away from x =0 can be approximated by Hele-Shaw flow in a thin viscous gap, where the viscous gap corresponds to the sublithospheric portion of the upper mantle with thickness λ
(Batchelor 1967; Acheson 1990; Furbish 1997).
![](./RH06_fig03.png)

We follow the $\mu = const$ assumption.

The general format of P is taken as:

$Pi\left(x, y\right) = \frac{A_i \left(x - x_i\right)}{\left(x - x_i\right)^2 + \left(y - y_i\right)^2}$ (A6)

The integral of $v_x$ over z is derived by a summation over 2n - 1 terms.
These terms are assymetric along y = 0 ($A_i = A_{-i}$).

$\int_{0}^{\lambda} v_x dz = \lambda \left(\frac{v_t + v_m}{2}\right) + \sum_{i=-(n-1)}^{n-1}\frac{\left(x - x_i\right)^2 - \left(y - y_i\right)^2}{\left[\left(x - x_i\right)^2 + \left(y - y_i\right)^2\right]^2} \left(\frac{A_i \lambda^3}{12 \mu} \right)$  (A8)

(The eq in the paper is wrong about the power on the bottom)

The the coefficients $A_i$ are solved by the constraint from an inversion

$\int_{0}^{\lambda} v_x dz = v_r \lambda,  (x = 10 km, y = y_i)$

In [ ]:
# initiate the class and assign an initial solution
Hele_Shaw = RH06.HELE_SHAW(Vt, Vm, Vr, L, lbd, mu, n_hs)
As = 0.1 * np.ones(n_hs) * mu / lbd**2.0 * (Vt + Vm) # initial guess
Hele_Shaw.AssignA(As)

In [ ]:
# plot the result with the initial guess
x = 10e3 # 10 km
ym = 4*a
fig, ax = plt.subplots()
Hele_Shaw.PlotVxdz(x, ym, ax=ax)
ax.set_xlim([0, 4.0])

In [ ]:
J = Hele_Shaw.Jacobi_slab(x)
# J = Hele_Shaw.Jacobi_full(x)

In [ ]:
x = 10e3 # 10 km
weight_inf = 10.0  # 0.0 means the infinite point is not included in the inversion
n_iter = 200
tolerance = 1e-6
Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd(x)
# Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd_full(x, weight_inf=weight_inf)
residuals = []

Vx_dzs_nd_std = np.ones(n_hs)  # values on the surface of the slab
# Vx_dzs_nd_std = np.ones(n_hs+1)  # values on the surface of the slab
# Vx_dzs_nd_std[n_hs] = weight_inf * lbd * (Vt + Vm) / 2.0  # values to the infinity x
    
# residuals  
vx_dzs_diff = Vx_dzs_nd_std - Vx_dzs_nd
residual = np.linalg.norm(vx_dzs_diff, ord=np.inf)
residuals.append(residual)

for i in range(n_iter):

    if residual < tolerance:
        break

    # jacobian 
    J = Hele_Shaw.Jacobi_slab(x)
    # J = Hele_Shaw.Jacobi_full(x)
    J1 = np.dot(J.transpose(), J)
    # J1 = np.dot(J, J.transpose())
    J1_inv = np.linalg.inv(J1)

    # apply iteration
    d_As = np.dot(J1_inv, np.dot(J.transpose(), vx_dzs_diff))
    As += d_As
    Hele_Shaw.AssignA(As)
    # Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd(x)
    Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd(x)
    
    # residuals  
    vx_dzs_diff = Vx_dzs_nd_std - Vx_dzs_nd
    residual = np.linalg.norm(vx_dzs_diff, ord=np.inf)
    residuals.append(residual)

# export residual and history
print("Final residual: ", residual)

if residual < tolerance:
    # convergence
    end_i = i
else:
    # inconvergence
    end_i = i + 1
fig, ax = plt.subplots()
ax.semilogy(range(end_i + 1), residuals)

In [ ]:
# replicate the results in Figure A1
x = 10e3 # 10 km
ym = 4*a
xm = 3*a

fig = plt.figure(tight_layout=True, figsize=(6, 10))
gs = gridspec.GridSpec(2,1)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])

Hele_Shaw.PlotVxdz(x, ym, ax=ax1)
ax1.set_xlim([0.0, ym/a])

Hele_Shaw.PlotCenterPressureX(xm, ax=ax2, x_nd=x)
ax2.set_xlim([0.0, xm/a])

In [ ]:
z0 = 100e3
fig, ax = plt.subplots()

h = Hele_Shaw.PlotPressureOnSurface(ax=ax, vmin=-0.3, vmax=0.3)
fig.colorbar(h, ax=ax, label='P [Mpa]')

xxs, yys, vvxs, vvys = Hele_Shaw.PlotVelocityOnSurface(z0, ax=ax)
fig.suptitle("Flow at z = %.2f km" % (z0 / 1e3))